In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("df.csv")

In [ ]:
pip install datasets

In [9]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
# Map sentiment labels to 0 (negative), 1 (neutral), 2 (positive)
label_map = {0: 0, 1: 1, 2: 2}
df["sentiment"] = df["sentiment"].map(label_map)

# Split into train & test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"], df["sentiment"], test_size=0.1, random_state=42
)

# Load DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize texts with max_length=64
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=64)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=64)

# Convert to PyTorch tensors
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, list(train_labels))
val_dataset = SentimentDataset(val_encodings, list(val_labels))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
model.gradient_checkpointing_enable()  # Saves memory

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,  # Reduce batch size for memory efficiency
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # Adjust based on performance
    fp16=True,  # Mixed precision (works on GPU)
    save_total_limit=1,  # Saves only 1 checkpoint to save disk space
    logging_dir="./logs",
    logging_steps=50,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kareenayadav343 (kareenayadav343-harcourt-butler-technical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.727000,0.623862
2,0.673100,0.836999
3,0.348900,1.018022


TrainOutput(global_step=11886, training_loss=0.5667679705002804, metrics={'train_runtime': 1153.7908, 'train_samples_per_second': 41.204, 'train_steps_per_second': 10.302, 'total_flos': 590413585203936.0, 'train_loss': 0.5667679705002804, 'epoch': 3.0})

In [ ]:
# Save the trained model
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

In [ ]:
from google.colab import files
import shutil

# Create a zip file of the saved model directory
shutil.make_archive('sentiment_model', 'zip', 'sentiment_model')

# Download the zip file
files.download('sentiment_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install torch transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch

# Load the saved model and tokenizer
model_path = "./sentiment_model"  # Path to the saved model directory
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)


In [ ]:
def predict_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)

    # Make prediction
    with torch.no_grad():  # No need to track gradients during inference
        outputs = model(**inputs)

    # Get the predicted class (index of the maximum logit)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()

    return predicted_class_id


In [ ]:
# Mapping of class labels to sentiment
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

def predict_sentiment_label(text):
    predicted_class_id = predict_sentiment(text)
    return label_map.get(predicted_class_id, "Unknown")

# Example usage
user_input = input("Enter a text for sentiment analysis: ")
prediction = predict_sentiment_label(user_input)
print(f"Predicted Sentiment: {prediction}")


Enter a text for sentiment analysis: i hate ml
Predicted Sentiment: Negative


In [ ]:
# Example usage
user_input = input("Enter a text for sentiment analysis: ")
prediction = predict_sentiment_label(user_input)
print(f"Predicted Sentiment: {prediction}")

Enter a text for sentiment analysis: The decision has been made based on the available information.
Predicted Sentiment: Neutral


In [ ]:
# Example usage
user_input = input("Enter a text for sentiment analysis: ")
prediction = predict_sentiment_label(user_input)
print(f"Predicted Sentiment: {prediction}")

Enter a text for sentiment analysis: i like ml
Predicted Sentiment: Positive
